# MULTIVARIATE FORECASTING USING SIMPLE & DEEP RECURRENT NEURAL NETWORKS

_**Multivariate time-series forcasting using Simple & Deep Recurrent Neural Networks (RNNs).**_

The following experiment considers Chicago Transit Authority (CTA) dataset (https://data.cityofchicago.org/) containing daily bus and rail ridership. The data from January 01, 2001 through August 1, 2024 was considered in this experiment.

In [ ]:
# Imports required packages

import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

## Retrieving Data

In [ ]:
# Loads the dataset
ridership = pd.read_csv(
    "./../data/CTA-Ridership-Daily_Boarding_Totals_20240829.csv", 
    parse_dates=["service_date"]  # To consider column as a date coumn if it contains date represented as text
)

# Shows the dataset
display(ridership)

Note: Attribute value **W**, **A** ans **U** in column **day_type** represent **Weekday**, **Saturday** and **Sunday/Holiday**, respectively.

In [3]:
# Write code to set the column 'service_date' as by calling DataFrame's set_index method
# and passing column name as argument. Also set `True` against `inplace` parameter to make
# changes right in the same DataFrame instead returning another copy

# ...

In [4]:
# Write code to sorts dataset (in ascending order) by set index in the above step to make index monotonic
# by calling DataFrame's `sort_index` method passing argument `True` to its `inplace` parameter.
# [Note that it would be a requirement for slicing DataFrame with a datetime type based index.]

# ...

In [5]:
# Write code to drop the calculated column "total_rides" by calling DataFrame's `drop` method
# passing the name of the column as parameter. Also pass second axis i.e. 1 to its `axis`
# parameter to indicate deletion to take from which dimention, and `True` to its `inplace`
# parameter to make changes right in the same DataFrame instead returning another copy
# [Note that this column is not required for modeling as this is just element-wise addition
# between columns "bus" and "rail_boardings".]

# ...

## Data Analysis

In [ ]:
# Plots the bus and rail ridership over a few months in 2019
ridership["2019-01":"2019-06"].plot(grid=True, marker=".", figsize=(10, 3))

Look the the figure above and observe for any repeating patterns such as weekly seasonality for both variables - bus and rail.

## Data Preprocessing

**Prepares Datasets for Modeling**

Prepares dataset with multiple features as input for modeling

In [ ]:
# Write code to create a new DataFrom taking column `bus` and `rail_boardings` from the `ridership` DataFrame
# Also, scale down all the values from milions to between 0 and 1 by dividing them by 1e6.
ridership_multivar = # ...

# Considering next day could affect the present day's ridership, write code to
# creates a feature on the next day (as type 'W', 'A' and 'U'), by calling
# `shift` method on the DataFrame series on column `day_type` and passing -1 into it.
ridership_multivar["next_day_type"] = # ...

# Write code to one-hot encode newly added categorical feature `next_day_type`
# by calling `pd.get_dummies` method passing new DataFrame series and `int` as `dtype`
ridership_multivar = # ...

# Shows the encoded multivariate dataset
display(ridership_multivar.head())

In [ ]:
# Checks the shape of the multivariate dataset
# ...

Check the increased number of columns in the new DataFrame.

In [9]:
# Write code to split the DataFrame into three representing split for training, validation and testing, respectively.
# Train set should contain data from 2016-01 to 2018-12 (3 years).
# Validation set should contain data from 2019-01 to 2019-06 (6 months)
# Test set should contain data from 2019-07 onwards.

multivar_train = # Write code
multivar_val = # Write code
multivar_test = # Write code

In [ ]:
# Prepares TensorFlow specific datasets

seq_length = 56    # sequence of 56 days of ridership data on which prediction for next day is made on

tf.random.set_seed(42)    # Sets global random seed for operations that rely on a random seed

train_set = tf.keras.utils.timeseries_dataset_from_array(
    # Convert train set DataFrame to numpy array by calling its `to_numpy()` method,
    # Set `targets` as `rail_boardings` series offset by `seq_length`
    # Set `sequence_length` to `seq_length` as length of output sequence
    # Set `batch_size` to 32 as number of sequences in each batch
    # Set `True` to 'shuffle` to shuffle the output sequences [required only for training]
    # Set 42 to `seed` as random seed for shuffling [required only if shuffling is set to True]
)

# Similarly, prepares validation and test Tensor data set from respective DataFrame 
# (without shuffling as it is not required for validation and testing)

val_set = tf.keras.utils.timeseries_dataset_from_array(
    # ...
    # ...
    # ...
    # ...
)

test_set = tf.keras.utils.timeseries_dataset_from_array(
    # ...
    # ...
    # ...
    # ...
)

## Modeling
_Forecasting next days's rail ridership based (only) on rail ridership [multiple variables as input] of the past 8 weeks (56 days) using simple and deep RNNs._

### Multivariate Forecasting using a Single RNN

In [11]:
# Resets all the keras states
tf.keras.backend.clear_session()

tf.random.set_seed(42)


# Write code to  create a sequential model calling `tf.keras.Sequential` method
# passing the list of the following layers.
# First layer is a `tf.keras.layers.Input` layer that takes a tuple as `shape`
# The first element in the tuple should be `None` to indicate RNN to accept 
# input sequence of any length, and 5 as second element to indicate single target variable.
# Second layer is `tf.keras.layers.SimpleRNN` that takes 32 as output recurrent units.
# Third ir output layer is `tf.keras.layers.Dense` that takes 1 as output unit as there 
# is only one target variable to predict.

multivar_simple_rnn = #...

In [12]:
# Compiles it with specific loss function, optimizer and metric

# Write code to compile the model by calling `multivar_simple_rnn.compile` method passing
# "huber" as `loss` as this is a popular loss function for RNN for regression task.
# Set `optimizer` parameter to `tf.keras.optimizers.SGD` passing 0.05 to its parameter
# `learning_rate` and 0.9 to `momentum`. Pass a list with value "mae" to paramter `metrics`.

# ...

In [ ]:
# Fits the model

# Write code to fit the model by calling `multivar_simple_rnn.fit` and passing the following
# arguments to its parameters
# Train TensorFlow data set as the first parameter
# Validation TensorFlow data set to the parameter `validation_data`
# 500 to paramter `epochs`
# A list containing an `tf.keras.callbacks.EarlyStopping` object to parameter `callbacks`.
# Pass the following argumements to `tf.keras.callbacks.EarlyStopping` to initialize.
# "val_mae" to `monitor` (to check for a specific metric).
# 50 to `patience` (to wait before stopping traning if validation performance does not improve)
# `True` to `restore_best_weights` (to return best performing model)

history = # ...

In [ ]:
# After training, model gets evaluated against validation data

# Write code to pass validation TensorFlow data set to `multivar_simple_rnn.evaluate` to get
# validation performance over loss and MSE.

val_loss, val_mae = # ...

print(f"\nValidation loss: {val_loss * 1e6:.4f}, MAE of the multivariate simple RNN: {val_mae * 1e6:.4f}")

Validation MAE for multivariate simple RNN is 23270 where the the same performance was 29316 for its univariate counterpart.

### Multivariate Forecasting using a Deep RNN

In [19]:
tf.keras.backend.clear_session()    # Resets all the keras states

tf.random.set_seed(42)

# Creates a Deep RNN with multiple layers of simple RNN each with 32 recurrent neurons 
# followed by a dense output layer with one output neuron

# Now, create a sequential model once again by referring to the same steps
# already mentioned above. Refer to the following changes to be made against the 
# above approach.
# Instead of just one layer of `tf.keras.layers.SimpleRNN`, there will be total
# three such consecutive layers. First two layers of `tf.keras.layers.SimpleRNN`
# should also be passed `True` to its parameter `return_sequences` to indicate the layer
# to return a sequence (instead of a vector) against an input sequence.
# Don't set the same in the 3rd RNN layer to indicate expectation of a vector
# (instead of sequence) against an input sequence.

multivar_deep_rnn = # ...

In [20]:
# Compiles the model

# Write code to compile the model `multivar_deep_rnn` referring to the same 
# instruction provided above. Arguments will remain the same.

# ...


In [ ]:
# Fits the model

# Write code to fit the model `multivar_deep_rnn` referring to the same 
# instruction provided above. Arguments will remain the same.

history = # ...

In [ ]:
# After training, model gets evaluated against validation data

# Write code to evaluate the model `multivar_deep_rnn` referring to the same 
# instruction provided above. Arguments will remain the same.

val_loss, val_mae = # ...

print(f"\nValidation loss: {val_loss * 1e6:.4f}, MAE of the multivariate deep RNN: {val_mae * 1e6:.4f}")

Compare the validation perform of these two models.

## Observations

- How was the new feature for next day was prepared? What was the importance of this feature?

- How was the input for the multivariate model different from the same of the univariate model?

- Did the multivariate model performed better than the univariate model for both simple and deep RNN based modeling approach? Analyse with metrics.